In [ ]:
%matplotlib widget
%store -r path

''' set to True the values you want to plot
If the criteria does not exist it will not be plotted'''

FORCES = True
DISPLACEMENTS = True

plot_delay = 2.5     #in seconds, 2.5 is WB Mech default

In [ ]:
import shutil
import os
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

In [ ]:
def animate(i):
    shutil.copyfile(path,'file.gst')
    tree = ET.parse('file.gst')
    root = tree.getroot()
    global FORCES, DISPLACEMENTS
    f_crit = False
    u_crit = False
    
    
    for child in root[0][0]:
        if str(child.text) == 'Cum Iter':
            cum_iter = int(child.attrib["ID"])
        elif str(child.text) == 'F   CRIT':
            f_crit = int(child.attrib["ID"])
        elif str(child.text) == 'F    L2 ':
            f_norm = int(child.attrib["ID"])
        elif str(child.text) == 'U   CRIT':
            u_crit = int(child.attrib["ID"])
        elif str(child.text) == 'U    INF':
            u_inf = int(child.attrib["ID"])
        elif str(child.text) == 'Time':
            time = int(child.attrib["ID"])
    
    FORCES = FORCES * f_crit
    DISPLACEMENTS = DISPLACEMENTS * u_crit
    print(DISPLACEMENTS)
    data_line = ''
    
    for ls in range(0,len(root[0:])):
        for child in root[ls:][0]:
            data_line = data_line + child.text
    data_line = data_line.split('\n')

    cum_iter_counter = []
    force_criteria = []
    force_norm = []
    displacement_criteria = []
    displacement_norm = []
    
    for c in range(0,len(data_line)):
        if data_line[c] != '' and data_line[c] != ' ' :
            while '  ' in data_line[c]:
                data_line[c] = data_line[c].replace('  ', ' ')
            data_line_current = data_line[c].split(' ')
            cum_iter_counter.append(float(data_line_current[cum_iter]))
            if FORCES:
                force_criteria.append(float(data_line_current[f_crit]))
                force_norm.append(float(data_line_current[f_norm]))
            if DISPLACEMENTS:
                displacement_criteria.append(float(data_line_current[u_crit]))
                displacement_norm.append(float(data_line_current[u_inf]))
    
    cum_iter_counter = np.asarray(cum_iter_counter)
    force_criteria = np.asarray(force_criteria)
    force_norm = np.asarray(force_norm)
    displacement_criteria = np.asarray(displacement_criteria)
    displacement_norm = np.asarray(displacement_norm)
    current_time = float(data_line_current[time])

    plt.cla()
    plt.title('Time = ' + str(current_time) + ' [sec]', loc = 'center')
    plt.yscale("log")
    if FORCES:
        plt.plot(cum_iter_counter, force_criteria, label = 'Force Crit')
        plt.plot(cum_iter_counter, force_norm, label = 'F Norm')
    if DISPLACEMENTS:
        plt.plot(cum_iter_counter, displacement_criteria, label = 'U Crit')
        plt.plot(cum_iter_counter, displacement_norm, label = 'U Norm')
    plt.legend(loc = 'upper left')

In [ ]:
ani = FuncAnimation(plt.gcf(), animate, interval = plot_delay*1000)    
plt.show()